In [1]:
import pandas as pd
# from mtl_data_to_text import get_justification_statement
from evaluate import load
import torch

In [2]:
data_df = pd.read_pickle('./data/data_verdict.pkl')

In [3]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,clean_claim_triples,main_text_triples,clean_main_text_triples,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,verdict,verdict_actual,clean_verdict
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...",-1,0,-1
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[(Shawnee County, located in the administrativ...","[(KSNT, located in the administrative territor...","[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...",-1,1,-1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...",-1,-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substances),...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...",-1,1,-1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...",-1,1,-1


In [4]:
def get_justification(row, cleaned=True):
    if cleaned:
        claim_evidences = row['cleaned_claim_evidence_1']
    else:
        claim_evidences = row['claim_evidence_1']
    claims = list(claim_evidences.keys())

    if cleaned:
        entailment_scores = row['cleaned_entailment_scores_1']
    else:
        entailment_scores = row['entailment_scores_1']
    index_values = [-1, 1, 0]

    justifications = []

    for i in range(len(claims)):
        which_verdict = entailment_scores[i].argmax()
        evidence = claim_evidences[claims[i]][0]
        claim = claims[i]
  
        justification = get_justification_statement(claim, evidence, index_values[which_verdict])

        justifications.append(justification)

    justification_mtl = ' '.join(justifications)
    
    if cleaned:
        row['cleaned_justification_mtl'] = justification_mtl
    else:
        row['justification_mtl'] = justification_mtl

    if row.name%100==0:
        print(row.name)
    
    return row


In [5]:
data_df = data_df.apply(lambda row : get_justification(row, cleaned=False), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [10]:
data_df = data_df.apply(lambda row : get_justification(row, cleaned=True), axis=1)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200


In [11]:
data_df

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,clean_main_text_triples,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,verdict,verdict_actual,clean_verdict,justification_mtl,cleaned_justification_mtl
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...",-1,0,-1,A baby died at an unnamed medical facility bec...,A baby died at an unnamed medical facility bec...
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...",-1,1,-1,Bat from Shawnee County is not caused by rabie...,Topeka is not located in Shawnee County. Bat f...
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...",-1,-1,-1,Germany banned pork from school canteen becaus...,Germany banned pork from school canteen becaus...
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...",-1,1,-1,Coronavirus is found in Canada and is unrelate...,Coronavirus is found in Canada and is unrelate...
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...",-1,1,-1,"The Wayne National Forest is not a forest, bec...",The Wayne National Forest is not an instance o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,3295,Christmas miracle for Florida dog whose heart ...,NaN,A Florida man was hoping for a Christmas mirac...,Associated Press,The heart of Gerald Ford’s 1-year-old French b...,https://www.gainesville.com/news/20200102/holi...,true,"Animals, Heal

In [12]:
print(data_df.iloc[5]['claim'])
print(data_df.iloc[5]['explanation'])
print(data_df.iloc[5]['label'])
print(data_df.iloc[5]['cleaned_justification_mtl'])
print(data_df.iloc[5]['justification_mtl'])

School closures will have little impact on COVID-19 control, review finds.
School closures do not tend to help contain the spread of infections during outbreaks of disease such as COVID-19, but will have a big impact on how societies restart after lockdown, scientists said on Monday.
true
The SARS-COV2 coronavirus caused a school closure. SARS-COV2 coronavirus was the cause of the COVID-19 pandemic, which has been under control.
The SARS-COV2 coronavirus caused school closures as part of COVID-19 control. The SARS-COV2 coronavirus caused a school closure. SARS-COV2 coronavirus was the cause of the COVID-19 pandemic, which has been under control.


In [13]:
data_df.to_pickle('./data/data_justifications.pkl')

In [5]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [2]:
data_df = pd.read_pickle('./data/data_justifications.pkl')

In [3]:
bertscore = load("bertscore")
rouge = load('rouge')
sacrebleu = load('sacrebleu')

def get_metrics(predictions, references, references_sb):
    results = {}    
    results["bertscore"] = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
    results["rouge"] = rouge.compute(predictions=predictions, references=references)
    results["sacrebleu"] = sacrebleu.compute(predictions=predictions, references=references_sb)

    return results

In [4]:
cleaned_justifications_mtl = data_df['cleaned_justification_mtl'].tolist()
justifications_mtl = data_df['justification_mtl'].tolist()
references = data_df['explanation'].tolist()
references_sb = [[reference] for reference in references]

In [8]:
cleaned_results_mtl = get_metrics(cleaned_justifications_mtl, references, references_sb)
print(cleaned_results_mtl)
results_mtl = get_metrics(justifications_mtl, references, references_sb)
print(results_mtl)

{'bertscore': {'precision': [0.7247609496116638, 0.6983183026313782, 0.7488046884536743, 0.7768673896789551, 0.7681998610496521, 0.7790247201919556, 0.7650513648986816, 0.8127774000167847, 0.7943344116210938, 0.7244381904602051, 0.8095958828926086, 0.7787196636199951, 0.8236089944839478, 0.7209125757217407, 0.7029904127120972, 0.6065937280654907, 0.7998002171516418, 0.7868058085441589, 0.7056461572647095, 0.8628942370414734, 0.751471221446991, 0.7450829744338989, 0.8059003353118896, 0.8418734073638916, 0.6831461191177368, 0.7202759981155396, 0.7820054292678833, 0.7575094103813171, 0.8117629885673523, 0.7919208407402039, 0.7299546003341675, 0.7873961329460144, 0.8431805372238159, 0.7479816675186157, 0.7692508101463318, 0.7351905107498169, 0.7673202157020569, 0.6922205686569214, 0.7640062570571899, 0.7475154399871826, 0.7300999164581299, 0.7899269461631775, 0.7072732448577881, 0.7609482407569885, 0.7342864871025085, 0.8317437171936035, 0.809161901473999, 0.6947947144508362, 0.73091876506

In [9]:
mtl_precision = sum(results_mtl['bertscore']['precision']) / len(results_mtl['bertscore']['precision'])
mtl_recall = sum(results_mtl['bertscore']['recall']) / len(results_mtl['bertscore']['recall'])
mtl_f1score = sum(results_mtl['bertscore']['f1']) / len(results_mtl['bertscore']['f1'])

print(f"BERTScore Precision: {mtl_precision}, Recall: {mtl_recall}, F1-score: {mtl_f1score}")

BERTScore Precision: 0.7594125883662876, Recall: 0.7160805005331023, F1-score: 0.7358768237162181


In [10]:
cleaned_mtl_precision = sum(cleaned_results_mtl['bertscore']['precision']) / len(cleaned_results_mtl['bertscore']['precision'])
cleaned_mtl_recall = sum(cleaned_results_mtl['bertscore']['recall']) / len(cleaned_results_mtl['bertscore']['recall'])
cleaned_mtl_f1score = sum(cleaned_results_mtl['bertscore']['f1']) / len(cleaned_results_mtl['bertscore']['f1'])

print(f"BERTScore Precision: {cleaned_mtl_precision}, Recall: {cleaned_mtl_recall}, F1-score: {cleaned_mtl_f1score}")

BERTScore Precision: 0.7698669622471438, Recall: 0.7149430643432876, F1-score: 0.7400893380601099


In [11]:
print(cleaned_results_mtl['rouge'])
print(results_mtl['rouge'])

{'rouge1': 0.1851472957466751, 'rouge2': 0.04416458566517716, 'rougeL': 0.13843662539500953, 'rougeLsum': 0.13853913000084367}
{'rouge1': 0.18584180969824127, 'rouge2': 0.041679921358179536, 'rougeL': 0.14019694337193406, 'rougeLsum': 0.14030556919339396}


In [12]:
print(cleaned_results_mtl['sacrebleu'])
print(results_mtl['sacrebleu'])

{'score': 0.6131975872914872, 'counts': [22547, 4054, 1153, 420], 'totals': [76698, 74246, 71794, 69342], 'precisions': [29.39711596130277, 5.460226813565714, 1.6059837869459843, 0.6056935190793459], 'bp': 0.17346947509517147, 'sys_len': 76698, 'ref_len': 211054}
{'score': 1.1458607457315961, 'counts': [28186, 4965, 1374, 489], 'totals': [107583, 105131, 102679, 100227], 'precisions': [26.19930658189491, 4.722679323891145, 1.3381509364134827, 0.48789248406118113], 'bp': 0.3822125743176673, 'sys_len': 107583, 'ref_len': 211054}


In [13]:
predictions = ["My name is Prahlad, I study at Columbia"]
references = ["My university is Columbia, I am Prahlad"]
results = sacrebleu.compute(predictions=predictions, references=references, tokenize='none', lowercase=True, use_effective_order=True)
print(results)

{'score': 7.267884212102741, 'counts': [3, 0, 0, 0], 'totals': [8, 7, 6, 5], 'precisions': [37.5, 7.142857142857143, 4.166666666666667, 2.5], 'bp': 1.0, 'sys_len': 8, 'ref_len': 7}


In [14]:
def get_stats(data_df, columns):

    for column in columns:
        data_df[f'{column}_tokens'] = data_df[column].apply(lambda n: len(str(n).split()))

get_stats(data_df, columns=['claim', 'main_text', 'explanation'])

In [15]:
small_just_df = data_df[(data_df['main_text_tokens'] < 100) & (data_df['main_text_tokens'] > 10) ]

In [16]:
small_just_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,cleaned_entailment_scores_1,entailment_scores_1,verdict,verdict_actual,clean_verdict,justification_mtl,cleaned_justification_mtl,claim_tokens,main_text_tokens,explanation_tokens
36,5583,Alabama: Case of meningococcal disease in Jack...,NaN,The Alabama Department of Public Health’s Immu...,,"In a news release Tuesday, the department said...",,true,"Immunizations, Meningitis, Health, Alabama, Pu...","[{'head': 'Alabama', 'type': 'contains adminis...",...,"[[-0.47511193, -0.49882457, 0.72487426], [-0.1...","[[-0.77345294, -0.0044897804, 0.6338379], [-0....",1,1,1,"Scottsboro is located in Jackson County, Alaba...","Scottsboro is located in Jackson County, Alaba...",8,90,34
121,40135,Little Lauren Renee Pingel was in a car accid...,"March 17, 2015",Pray for 3-year-old Lauren Renee Pingel in Ama...,Rich Buhler & Staff,This story is true… and the latest news is tha...,https://twitter.com/intent/tweet?text=Pray+for...,true,Prayers,"[{'head': 'Little Lauren Renee Pingel', 'type'...",...,"[[-0.30718657, -0.6257052, 0.7170282], [0.6203...","[[-0.32555494, -0.60986596, 0.7225494], [-0.30...",1,1,1,Little Lauren Renee Pingel was born on Decembe...,"Lauren Renee Pingel was born in Amarillo, Texa...",32,69,9
189,7264,Zambia to vaccinate 1 million against cholera ...,NaN,Zambia is set to vaccinate one million people ...,,The World Health Organization says doses of th...,,true,"Health, Cholera, Zambia, Africa, Southern Afri...","[{'head': 'outbreak', 'type': 'has cause', 'ta...",...,"[[-0.6810705, -0.19560058, 0.7056086], [0.6290...","[[-0.6810705, -0.19560058, 0.7056086], [0.6290...",-1,1,-1,An outbreak of cholera is unrelated to acute d...,An outbreak of cholera is unrelated to acute d...,9,93,18
190,5834,State to labs: report all results of tests for...,NaN,Louisiana’s Department of Health says laborato...,,Louisiana has very high rates of all three dis...,http://ldh.la.gov/index.cfm/newsroom/detail/5162,true,"Health, Hepatitis, Syphilis, Louisiana, Public...","[{'head': 'tests', 'type': 'facet of', 'tail':...",...,"[[-0.8129388, 0.15043166, 0.56258404], [0.0264...","[[-0.8129388, 0.15043166, 0.56258404], [0.0264...",1,1,1,The CDC is unrelated to Centers for Disease Co...,The CDC is unrelated to Centers for Disease Co...,11,96,27
237,4952,Health care worker in Grant County has mumps.,NaN,The Grant County Health District has issued an...,,The health district says a vaccinated health c...,http://www.kxly.com/,true,"Health, Moses Lake, Mumps",[{'head': 'Health care worker in Grant County'...,...,"[[-0.61041945, -0.31841892, 0.72525686], [0.60...","[[0.59940296, -0.771737, -0.21245721], [-0.610...",-1,1,1,Samaritan Healthcare is located in Moses Lake ...,Samaritan Healthcare is located in Moses Lake ...,8,84,20


In [19]:
which_just = small_just_df.iloc[34]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['cleaned_justification_mtl'])
print(which_just['justification_mtl'])

Ohio officials say tick-borne Lyme Disease still a threat.
The state agency reports that 241 cases of Lyme disease have been reported in 60 of Ohio’s 88 counties this year. The number of cases has risen steadily the last five years, with 160 reported last year after only 93 in 2013. Lyme disease is passed to humans by deer ticks and can cause flu-like illnesses, muscle pain and headaches. A rash shaped like a bull’s-eye is often seen around tick bites. The Ohio Department of Natural Resources recommends people walk in the middle of trails and tuck in clothing to prevent bites.
The Ohio Department of Health says hunters and those trekking in wooded areas should use bug spray and take other precautions even in cold weather to prevent being infected with tick-borne Lyme disease.
true
[{'head': 'Lyme Disease', 'type': 'has cause', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'head': 'Lyme Disease', 'type': 'subclass of', 'tail': 'tick-borne', 'meta': {'spans': [[0, 128]]}}, {'hea

In [26]:
which_just = small_just_df.loc[189]
print(which_just['claim'])
print(which_just['main_text'])
print(which_just['explanation'])
print(which_just['label'])
print(which_just['claim_kg'])
print(which_just['main_text_kg'])
print(which_just['clean_claim_triples'])
print(which_just['clean_main_text_triples'])
print(which_just['claim_evidence_1'])
print(which_just['entailment_scores_1'])
print(which_just['verdict'])
print(which_just['verdict_actual'])
print(which_just['cleaned_justification_mtl'])
print(which_just['justification_mtl'])

Zambia to vaccinate 1 million against cholera amid outbreak.
The World Health Organization says doses of the cholera vaccine have been delivered to the impoverished southern African nation as fears spread in Lusaka. Zambia’s health ministry says 58 of the 63 countrywide deaths in the cholera outbreak that began in early October have occurred in the capital. Of the 2,672 cases across the country, 2,558 have been in Lusaka. Cholera is an acute diarrheal and bacterial infection caused by ingestion of contaminated food or water. Just hours after being infected, vomiting and diarrhea cause severe dehydration that can kill without rapid intervention.
Zambia is set to vaccinate one million people in its capital against cholera as a deadly outbreak grows.
true
[{'head': 'outbreak', 'type': 'has cause', 'tail': 'cholera', 'meta': {'spans': [[0, 128]]}}, {'head': 'outbreak', 'type': 'country', 'tail': 'Zambia', 'meta': {'spans': [[0, 128]]}}, {'head': 'vaccine', 'type': 'country', 'tail': 'Zambi